# Challenge
Use RBM to perform feature extraction on an image-based dataset that you find or create. If you go this route, present the features you extract and explain why this is a useful feature extraction method in the context you’re operating in. DO NOT USE either the MNIST digit recognition database or the iris data set. They’ve been worked on in very public ways very very many times and the code is easily available. (However, that code could be a useful resource to refer to)

In [1]:
import numpy as np
import pandas as pd

from zipfile import ZipFile
from sklearn.model_selection import train_test_split
from sklearn.neural_network import BernoulliRBM
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [2]:
!cd ../../data_sets/f & ls

30
Neural_Net_Challenge.ipynb
supervised_neural_network.ipynb
test_zip.zip
train_zip.zip


The system cannot find the path specified.


data source: https://www.kaggle.com/sirsolim/images-of-primitive-3d-objects-for-classification/

In [3]:
!cd ../../data_sets/3d_images/ & ls

features_csv.csv
features_npy.npy
labels_csv.csv
labels_npy.npy


In [4]:
data_df = pd.read_csv(r'../../data_sets/3d_images/features_csv.csv')
data_df.head()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.1443,0.1444,0.1445,0.1446,0.1447,0.1448,0.1449,0.1450,0.1451,0.1452
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
data_df.shape

(479999, 1600)

In [6]:
data2_df = data_df.astype('bool').iloc[0:50_000,:]

In [7]:
y = pd.read_csv(r'../../data_sets/3d_images/labels_csv.csv')
y = y[0:50_000]

In [8]:
y.head()

,0,0.1,1,0.2
0,0,0,1,0
1,0,0,1,0
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1


In [9]:
# Un-OHE y
y = y.idxmax(1)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(data2_df, y, test_size=0.33)

In [11]:
bern_rbm = BernoulliRBM(learning_rate=.05, n_iter=20, verbose=1)
bern_rbm.fit_transform(X_train, y_train)

[BernoulliRBM] Iteration 1, pseudo-likelihood = -90.30, time = 97.99s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -86.17, time = 95.96s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -75.29, time = 94.80s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -58.88, time = 96.44s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -71.90, time = 94.79s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -64.91, time = 94.17s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -51.41, time = 91.58s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -65.66, time = 92.37s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -60.63, time = 91.35s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -48.74, time = 90.81s
[BernoulliRBM] Iteration 11, pseudo-likelihood = -64.75, time = 91.78s
[BernoulliRBM] Iteration 12, pseudo-likelihood = -65.50, time = 98.64s
[BernoulliRBM] Iteration 13, pseudo-likelihood = -63.69, time = 97.02s
[BernoulliRBM] Iteration 14, pseudo-likelihood = -64.18, time = 100.43s
[BernoulliRBM]

array([[1.90842973e-10, 2.08342407e-09, 8.82563528e-25, ...,
        1.00000000e+00, 1.00000000e+00, 2.40925741e-32],
       [1.88086644e-32, 4.37945907e-13, 1.35322951e-35, ...,
        3.66342850e-04, 2.92492846e-12, 1.00000000e+00],
       [8.91398963e-16, 2.32838637e-13, 1.01012712e-13, ...,
        4.46702069e-15, 5.70037902e-42, 5.06828254e-17],
       ...,
       [1.37722131e-06, 3.33907053e-09, 8.60482560e-08, ...,
        1.00000000e+00, 1.00000000e+00, 1.63709353e-25],
       [1.63741264e-51, 4.23723984e-50, 6.11911202e-47, ...,
        4.26886849e-48, 8.59758057e-62, 1.21236233e-45],
       [7.80411690e-16, 9.99999989e-01, 2.59130368e-25, ...,
        3.01673607e-23, 1.97247149e-36, 1.00000000e+00]])

In [12]:
y_test.size

16500

In [16]:
raw_logit = LogisticRegression(multi_class='multinomial', tol=1e-2, solver='saga', C=150)
raw_logit.fit(X_train, y_train)
raw_logit_pred = raw_logit.predict(X_test)
classification_report(y_true=y_test, y_pred = raw_logit_pred)

'              precision    recall  f1-score   support\n\n           0       0.99      1.00      0.99      3981\n         0.1       0.98      0.98      0.98      4162\n         0.2       0.98      0.98      0.98      4239\n           1       0.98      0.99      0.99      4118\n\n    accuracy                           0.99     16500\n   macro avg       0.99      0.99      0.99     16500\nweighted avg       0.99      0.99      0.99     16500\n'

In [18]:
FE_logit = LogisticRegression(multi_class='multinomial', tol=1e-3, solver='saga', C=100)
FE_logit.fit(bern_rbm, y_train)

TypeError: float() argument must be a string or a number, not 'BernoulliRBM'

In [ ]:
classification_report(y_true=y, y_pred = FE_logit)